In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
a = requests.get("https://www.naver.com")
a

In [13]:
a.text?

doc = bs(a.text, 'html.parser')
doc

<Response [200]>

In [17]:
result = doc.select('li.ah_item > a.ah_a > span.ah_k')

In [21]:
result[0].text

'디아크'